In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle("df_cleaned.pkl")

In [3]:
df.head()

,Article,Title,Paragraphs,References_internal_clean,Paragraphs_cleaned
1,Article 1,Scope,This Regulation lays down uniform rules concer...,[460],This Regulation lays down uniform rules concer...
2,Article 2,Supervisory powers,For the purposes of ensuring compliance with t...,[],For the purposes of ensuring compliance with t...
3,Article 3,Application of stricter requirements by instit...,This Regulation shall not prevent institutions...,[],This Regulation shall not prevent institutions...
4,Article 4,Definitions,"1. For the purposes of this Regulation, the ...","[4, 2, 115, 25, 71, 301, 113, 1]","1. For the purposes of this Regulation, the ..."
5,Article 5,Definitions specific to capital requirements f...,"For the purposes of Part Three, Title II, the ...",[],"For the purposes of Part Three, Title II, the ..."


In [4]:
del df['Paragraphs']

In [5]:
def strtonumber(numbers):
    numbers = list(map(int, numbers))
    return numbers

In [6]:
df['References'] = df['References_internal_clean'].apply(strtonumber)

In [7]:
df.head()

,Article,Title,References_internal_clean,Paragraphs_cleaned,References
1,Article 1,Scope,[460],This Regulation lays down uniform rules concer...,[460]
2,Article 2,Supervisory powers,[],For the purposes of ensuring compliance with t...,[]
3,Article 3,Application of stricter requirements by instit...,[],This Regulation shall not prevent institutions...,[]
4,Article 4,Definitions,"[4, 2, 115, 25, 71, 301, 113, 1]","1. For the purposes of this Regulation, the ...","[4, 2, 115, 25, 71, 301, 113, 1]"
5,Article 5,Definitions specific to capital requirements f...,[],"For the purposes of Part Three, Title II, the ...",[]


In [8]:
frame = pd.DataFrame(columns = ['Article1', 'Article1_Paragraphs', 'Article2', 'Article2_Paragraphs'])
frame.index = frame.index + 1

In [9]:
articles1 = list()
paragraphs1 = list()
articles2 = list()
paragraphs2 = list()
label = list()
articles1b = list()
paragraphs1b = list()
articles2b = list()
paragraphs2b = list()
labelb = list()
for x in range (1, len(df)):
    numbers = df['References_internal_clean'][x]
    numbers = list(map(int, numbers))
    for y in range (1, len(df)):
        if y in numbers:
            articles1.append(df['Article'][x])
            paragraphs1.append(df['Paragraphs_cleaned'][x])
            articles2.append(df['Article'][y])
            paragraphs2.append(df['Paragraphs_cleaned'][y])
            label.append(1)
        else:
            articles1b.append(df['Article'][x])
            paragraphs1b.append(df['Paragraphs_cleaned'][x])
            articles2b.append(df['Article'][y])
            paragraphs2b.append(df['Paragraphs_cleaned'][y])
            labelb.append(0)  

In [10]:
crr_articles = { 'articles1': articles1,'paragraphs1': paragraphs1, 'articles2': articles2, 'paragraphs2': paragraphs2, 'label': label}

frame = pd.DataFrame(crr_articles, columns = ['articles1', 'paragraphs1', 'articles2', 'paragraphs2', 'label'])

In [11]:
crr_articles2 = { 'articles1': articles1b,'paragraphs1': paragraphs1b, 'articles2': articles2b, 'paragraphs2': paragraphs2b, 'label': labelb}

frame2 = pd.DataFrame(crr_articles2, columns = ['articles1', 'paragraphs1', 'articles2', 'paragraphs2', 'label'])

In [12]:
len(frame2)

272339

In [13]:
len(frame)

1190

In [14]:
frame2_sample = frame2.sample(n=1190)

In [15]:
len(frame2_sample)

1190

In [16]:
frames = [frame, frame2_sample]
result = pd.concat(frames)
len(result)

2380

In [17]:
result.head()

,articles1,paragraphs1,articles2,paragraphs2,label
0,Article 1,This Regulation lays down uniform rules concer...,Article 460,1. The Commission shall be empowered to adop...,1
1,Article 4,"1. For the purposes of this Regulation, the ...",Article 1,This Regulation lays down uniform rules concer...,1
2,Article 4,"1. For the purposes of this Regulation, the ...",Article 2,For the purposes of ensuring compliance with t...,1
3,Article 4,"1. For the purposes of this Regulation, the ...",Article 4,"1. For the purposes of this Regulation, the ...",1
4,Article 4,"1. For the purposes of this Regulation, the ...",Article 25,The Tier 1 capital of an institution consists ...,1


In [18]:
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [19]:
result['cosine_similarity'] = result.apply(lambda x: get_result(x['paragraphs1'], x['paragraphs2']), axis=1)

In [20]:
result.head()

,articles1,paragraphs1,articles2,paragraphs2,label,cosine_similarity
0,Article 1,This Regulation lays down uniform rules concer...,Article 460,1. The Commission shall be empowered to adop...,1,0.448342
1,Article 4,"1. For the purposes of this Regulation, the ...",Article 1,This Regulation lays down uniform rules concer...,1,0.482119
2,Article 4,"1. For the purposes of this Regulation, the ...",Article 2,For the purposes of ensuring compliance with t...,1,0.458057
3,Article 4,"1. For the purposes of this Regulation, the ...",Article 4,"1. For the purposes of this Regulation, the ...",1,1.000000
4,Article 4,"1. For the purposes of this Regulation, the ...",Article 25,The Tier 1 capital of an institution consists ...,1,0.555750


In [21]:
#Count common words
def common_words(x):
    q1, q2 = x
    return len(set(str(q1).lower().split()) & set(str(q2).lower().split()))

In [22]:
result['common_words'] = result[['paragraphs1', 'paragraphs2']].apply(common_words, axis=1)

In [23]:
result.head()

,articles1,paragraphs1,articles2,paragraphs2,label,cosine_similarity,common_words
0,Article 1,This Regulation lays down uniform rules concer...,Article 460,1. The Commission shall be empowered to adop...,1,0.448342,29
1,Article 4,"1. For the purposes of this Regulation, the ...",Article 1,This Regulation lays down uniform rules concer...,1,0.482119,57
2,Article 4,"1. For the purposes of this Regulation, the ...",Article 2,For the purposes of ensuring compliance with t...,1,0.458057,17
3,Article 4,"1. For the purposes of this Regulation, the ...",Article 4,"1. For the purposes of this Regulation, the ...",1,1.000000,1423
4,Article 4,"1. For the purposes of this Regulation, the ...",Article 25,The Tier 1 capital of an institution consists ...,1,0.555750,13


In [26]:
result.to_pickle("cosine_commonwords.pkl")

In [29]:
#Create Fuzzy features
import fuzzywuzzy
from fuzzywuzzy import fuzz
result['fuzz_qratio'] = result.apply(lambda x: fuzz.QRatio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1)
result['fuzz_WRatio'] = result.apply(lambda x: fuzz.WRatio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1) 
result['fuzz_partial_ratio'] = result.apply(lambda x: fuzz.partial_ratio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1) 
result['fuzz_partial_token_set_ratio'] = result.apply(lambda x: fuzz.partial_token_set_ratio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1)
result['fuzz_partial_token_sort_ratio'] = result.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1) 
result['fuzz_token_set_ratio'] = result.apply(lambda x: fuzz.token_set_ratio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1)
result['fuzz_token_sort_ratio'] = result.apply(lambda x: fuzz.token_sort_ratio(str(x['paragraphs1']), str(x['paragraphs2'])), axis=1)

C:\Users\skamuf\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [30]:
result.head()

,articles1,paragraphs1,articles2,paragraphs2,label,cosine_similarity,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,Article 1,This Regulation lays down uniform rules concer...,Article 460,1. The Commission shall be empowered to adop...,1,0.448342,29,3,47,5,100,8,49,4
1,Article 4,"1. For the purposes of this Regulation, the ...",Article 1,This Regulation lays down uniform rules concer...,1,0.482119,57,0,57,7,100,5,84,0
2,Article 4,"1. For the purposes of this Regulation, the ...",Article 2,For the purposes of ensuring compliance with t...,1,0.458057,17,1,57,39,100,28,87,1
3,Article 4,"1. For the purposes of this Regulation, the ...",Article 4,"1. For the purposes of this Regulation, the ...",1,1.000000,1423,100,100,100,100,100,100,100
4,Article 4,"1. For the purposes of this Regulation, the ...",Article 25,The Tier 1 capital of an institution consists ...,1,0.555750,13,1,57,52,100,21,100,1


In [31]:
result.to_pickle("cosine_commonwords_fuzzy.pkl")